In [142]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import glob
from datetime import date
from datetime import datetime
import math
import requests
from math import sin, cos, sqrt, atan2, radians

In [159]:
month = '10'
df = pd.read_csv(r'airbike\2019'+month+'.csv')
df = df[['tripduration', 'starttime', 'startstationid', 'startstationlatitude',
       'startstationlongitude', 'endstationid', 'endstationlatitude',
       'endstationlongitude', 'aqstation', 'aqdistance', 'o3']]
df = df[df['o3'] != '-']
df.head()

,tripduration,starttime,startstationid,startstationlatitude,startstationlongitude,endstationid,endstationlatitude,endstationlongitude,aqstation,aqdistance,o3
1,174,2019-10-01 00:00:00,3301,40.791956,-73.968087,3283,40.788221,-73.970416,CCNY,3.514977,0.022000000000000002
26,307,2019-10-01 00:00:00,3314,40.793770,-73.971888,3366,40.802117,-73.968181,CCNY,3.511314,0.022000000000000002
28,737,2019-10-01 00:00:00,3514,40.776700,-73.927631,3513,40.774645,-73.923706,Bronx - IS52,4.877662,0.022000000000000002
37,441,2019-10-01 00:00:00,3494,40.797911,-73.942300,3387,40.793434,-73.949450,CCNY,2.472288,0.022000000000000002
39,322,2019-10-01 00:00:00,3145,40.778627,-73.957721,3299,40.788130,-73.952060,CCNY,4.639821,0.022000000000000002


In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270262 entries, 1 to 2092564
Data columns (total 11 columns):
tripduration             270262 non-null int64
starttime                270262 non-null object
startstationid           270262 non-null int64
startstationlatitude     270262 non-null float64
startstationlongitude    270262 non-null float64
endstationid             270262 non-null int64
endstationlatitude       270262 non-null float64
endstationlongitude      270262 non-null float64
aqstation                270262 non-null object
aqdistance               270262 non-null float64
o3                       270262 non-null object
dtypes: float64(5), int64(3), object(3)
memory usage: 21.7+ MB


In [161]:
df[df['o3'] != '-'].shape

(270262, 11)

In [162]:
def merge_start_end_coords(lat1,lon1,lat2,lon2):
    return f'{lat1},{lon1}|{lat2},{lon2}'
df['coords'] = df.apply(lambda row: merge_start_end_coords(row['startstationlatitude'],row['startstationlongitude'],row['endstationlatitude'],row['endstationlongitude'],),axis = 1)
df.head()

,tripduration,starttime,startstationid,startstationlatitude,startstationlongitude,endstationid,endstationlatitude,endstationlongitude,aqstation,aqdistance,o3,coords
1,174,2019-10-01 00:00:00,3301,40.791956,-73.968087,3283,40.788221,-73.970416,CCNY,3.514977,0.022000000000000002,"40.7919557,-73.968087|40.7882213,-73.97041561"
26,307,2019-10-01 00:00:00,3314,40.793770,-73.971888,3366,40.802117,-73.968181,CCNY,3.511314,0.022000000000000002,"40.7937704,-73.971888|40.8021174,-73.9681805305"
28,737,2019-10-01 00:00:00,3514,40.776700,-73.927631,3513,40.774645,-73.923706,Bronx - IS52,4.877662,0.022000000000000002,"40.7767,-73.927631|40.774645,-73.92370600000002"
37,441,2019-10-01 00:00:00,3494,40.797911,-73.942300,3387,40.793434,-73.949450,CCNY,2.472288,0.022000000000000002,"40.797911,-73.9423|40.79343370000001,-73.94945003"
39,322,2019-10-01 00:00:00,3145,40.778627,-73.957721,3299,40.788130,-73.952060,CCNY,4.639821,0.022000000000000002,"40.77862688,-73.95772073|40.78813,-73.95206"


In [163]:
unique_trips = df['coords'].value_counts().rename_axis('coords').reset_index(name='counts')
unique_trips.head()

,coords,counts
0,"40.77140426,-73.9535166|40.77314236,-73.95856158",375
1,"40.77579376683666,-73.97620573639871|40.772828...",334
2,"40.779668090073116,-73.98093044757842|40.78293...",310
3,"40.7869946,-73.94164802|40.7839636,-73.9471672...",305
4,"40.779668090073116,-73.98093044757842|40.78524...",299


In [6]:
#u1 = unique_trips[20000:]
#u1.shape

(3447, 2)

In [7]:
def get_distance_map_api(lat1,long1, lat2, long2):
  #key from https://developer.mapquest.com/documentation/samples/directions/v2/route/
  r = requests.get('https://www.mapquestapi.com/directions/v2/route?key=lYrP4vF3Uk5zgTiGGuEzQGwGIVDGuy24&unit=k&from='+str(lat1)+'%2C'+str(long1)+'&to='+str(lat2)+'%2C'+str(long2)+'&outFormat=json&ambiguities=ignore&routeType=bicycle&doReverseGeocode=false&enhancedNarrative=false&avoidTimedConditions=true')
  return r.json()['route']['distance']
def get_distance(coords):
    coords = coords.split('|')
    startcoords = coords[0].split(',')
    endcoords = coords[1].split(',')
    return float(get_distance_map_api(startcoords[0],startcoords[1],endcoords[0], endcoords[1]))

In [9]:
#u1['tripdistance'] = u1['coords'].apply(lambda x: get_distance(x))
#u1.head()

c:\program files (x86)\python37-32\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,coords,counts,tripdistance
20000,"40.80083629999999,-73.9664492472|40.7805779901...",1,3.5583
20001,"40.8006721,-73.9349|40.778301,-73.9488134",1,2.9467
20002,"40.801694,-73.957145|40.72430527250332,-73.996...",1,10.7633
20003,"40.7973721,-73.97041192|40.7728384,-73.94989233",1,4.2487
20004,"40.7781314,-73.96069399|40.71850211,-73.98329859",1,9.3261


In [10]:
#u1.tripdistance.value_counts()

0.0000    18
4.3533     8
3.3330     6
3.6419     5
2.9113     5
          ..
8.4555     1
1.3116     1
4.6880     1
3.2074     1
7.0682     1
Name: tripdistance, Length: 2524, dtype: int64

In [11]:
#u1.to_csv(r'C:\Users\Jacobo\Desktop\citibike2019\coordsdistance\coordsdistance3.csv')

In [10]:
'''u1 = pd.read_csv(r'C:\Users\Jacobo\Desktop\citibike2019\coordsdistance\coordsdistance1.csv')
u2 = pd.read_csv(r'C:\Users\Jacobo\Desktop\citibike2019\coordsdistance\coordsdistance2.csv')
u3 = pd.read_csv(r'C:\Users\Jacobo\Desktop\citibike2019\coordsdistance\coordsdistance3.csv')
unique_trips_distance = pd.concat([u1,u2,u3])
unique_trips_distance.columns = ['index','coords','count','tripdistance']
unique_trips_distance.head()
'''


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 21-22: truncated \UXXXXXXXX escape (<ipython-input-10-88b01e8ea0c9>, line 7)

In [164]:
unique_trips_distance = pd.read_csv(r'C:\Users\Jacobo\Desktop\citibike2019\coordsdistance\coordsdistance.csv')

In [165]:
unique_trips_distance.shape

(68884, 12)

In [166]:
unique_trips.shape

(32578, 2)

In [167]:
def get_calculated_distance(coords):
    cordsplit = coords.split('|')
    if(cordsplit[0] == cordsplit[1]):
        return 0.0
    val = unique_trips_distance[unique_trips_distance['coords']==coords]
    if (len(val) == 1):
        return val.tripdistance.values[0]
    else:
        return None
unique_trips['tripdistance'] = unique_trips.coords.apply(lambda x: get_calculated_distance(x))
unique_trips.head()

,coords,counts,tripdistance
0,"40.77140426,-73.9535166|40.77314236,-73.95856158",375,0.5182
1,"40.77579376683666,-73.97620573639871|40.772828...",334,1.6013
2,"40.779668090073116,-73.98093044757842|40.78293...",310,0.4088
3,"40.7869946,-73.94164802|40.7839636,-73.9471672...",305,0.9254
4,"40.779668090073116,-73.98093044757842|40.78524...",299,0.7210


In [168]:
unique_trips.head()

,coords,counts,tripdistance
0,"40.77140426,-73.9535166|40.77314236,-73.95856158",375,0.5182
1,"40.77579376683666,-73.97620573639871|40.772828...",334,1.6013
2,"40.779668090073116,-73.98093044757842|40.78293...",310,0.4088
3,"40.7869946,-73.94164802|40.7839636,-73.9471672...",305,0.9254
4,"40.779668090073116,-73.98093044757842|40.78524...",299,0.7210


In [169]:
no_distance= unique_trips[unique_trips['tripdistance'].isnull()]
no_distance.shape

(3376, 3)

In [170]:
no_distance['tripdistance'] = no_distance['coords'].apply(lambda x: get_distance(x))

c:\program files (x86)\python37-32\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [171]:
no_distance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3376 entries, 163 to 32577
Data columns (total 3 columns):
coords          3376 non-null object
counts          3376 non-null int64
tripdistance    3376 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 92.3+ KB


In [172]:
unique_trips_distance = pd.concat([unique_trips_distance,no_distance])
unique_trips = unique_trips[unique_trips['tripdistance'].isnull() == False]
unique_trips = pd.concat([unique_trips,no_distance])
unique_trips_distance.to_csv(r'C:\Users\Jacobo\Desktop\citibike2019\coordsdistance\coordsdistance.csv')
unique_trips.head()

c:\program files (x86)\python37-32\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,coords,counts,tripdistance
0,"40.77140426,-73.9535166|40.77314236,-73.95856158",375,0.5182
1,"40.77579376683666,-73.97620573639871|40.772828...",334,1.6013
2,"40.779668090073116,-73.98093044757842|40.78293...",310,0.4088
3,"40.7869946,-73.94164802|40.7839636,-73.9471672...",305,0.9254
4,"40.779668090073116,-73.98093044757842|40.78524...",299,0.7210


In [173]:
df_airbike = df.merge(how='left', right=unique_trips, left_on='coords', right_on='coords').drop(['coords','counts'], axis=1).fillna("")
df_airbike.head()

,tripduration,starttime,startstationid,startstationlatitude,startstationlongitude,endstationid,endstationlatitude,endstationlongitude,aqstation,aqdistance,o3,tripdistance
0,174,2019-10-01 00:00:00,3301,40.791956,-73.968087,3283,40.788221,-73.970416,CCNY,3.514977,0.022000000000000002,0.9640
1,307,2019-10-01 00:00:00,3314,40.793770,-73.971888,3366,40.802117,-73.968181,CCNY,3.511314,0.022000000000000002,1.0927
2,737,2019-10-01 00:00:00,3514,40.776700,-73.927631,3513,40.774645,-73.923706,Bronx - IS52,4.877662,0.022000000000000002,0.4152
3,441,2019-10-01 00:00:00,3494,40.797911,-73.942300,3387,40.793434,-73.949450,CCNY,2.472288,0.022000000000000002,1.4677
4,322,2019-10-01 00:00:00,3145,40.778627,-73.957721,3299,40.788130,-73.952060,CCNY,4.639821,0.022000000000000002,1.4484


In [174]:
df_airbike.to_csv(r'\airbikedistance\abd2019'+month+'.csv')